In [ ]:
pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 696 kB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [ ]:
import pandas as pd
import seaborn as sns
import plotly.express as px
from copy import copy
import matplotlib.pyplot as plt
import numpy as np
import plotly.figure_factory as ff
import plotly.graph_objects as go
import yfinance as yf

In [ ]:
tickers = "VIRT MSFT KO IBM BRK-A SHEL BTC-USD"  # please do it with spaces "AAPL MSFT SPY" example
allocation = [0.145,0.2,0.145,0.15,0.19,0.145,0.025] # proportion of capital in each stock/ticker, needs to add up to 1
initial_capital = 100000 # starting portfolio value
tickers_pagar = "ASIANPAINT.NS HDFCBANK.NS REDINGTON.NS HINDUNILVR.NS PIDILITIND.NS"
tickers_pagar_REIT = "BIRET.BO EMBASSY.BO MINDSPACE.BO"
allocation_pagar = [0.3,0.4,0.3]

ticker_pagar_REIT = ""
# Make sure SPY or the country benchmark is their always last
Market = 'SPY'
risk_free_rate = 1.069 # should be for country where that stock is based. Not sure if yfinance has this
start_date = "2022-10-31" # YYYY-MM-DD
end_date =  "2022-11-04"   # YYYY-MM-DD 
ticker_list = tickers_pagar_REIT.split(" ") #storing tickers seperately for reference

ticker_list

['BIRET.BO', 'EMBASSY.BO', 'MINDSPACE.BO']

In [ ]:
def create_dataset(ticker_list):
  raw_data = yf.download(ticker_list[0], interval='1d', start = start_date, end = end_date)
  raw_data['{}'.format(ticker_list[0])] = (raw_data['Open'] + raw_data['Close'])/2
  #ticker column is the mid price


  output_data = raw_data.filter(['Date','{}'.format(ticker_list[0])])
  if len(ticker_list) < 2:
    return output_data

#looping to add all each tickers data
  for i in range(1,len(ticker_list)):
    data = yf.download(ticker_list[i], interval='1d', start = start_date, end = end_date)
    data['{}'.format(ticker_list[i])] = (data['Open'] + data['Close'])/2
    output_data['{}'.format(ticker_list[i])] = data['{}'.format(ticker_list[i])]

  return output_data

#function to create a dataset with returns
def normalize(df):
  x = df.copy()
  for i in x.columns[0:]:
    x[i] = x[i]/x[i][0]
  return x

,BIRET.BO,EMBASSY.BO,MINDSPACE.BO
Date,,,


In [ ]:
prices_df = create_dataset(ticker_list)
returns_df = normalize(prices_df)

portfolio_df = returns_df*allocation_pagar*initial_capital # initialising portofolio with correct weights and capital 
portfolio_df['Total'] = portfolio_df.sum(axis=1)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [ ]:
portfolio_df

,BIRET.BO,EMBASSY.BO,MINDSPACE.BO,Total
Datetime,,,,
2022-10-31 09:17:00,30000.000000,40000.000000,30000.000000,100000.000000
2022-10-31 09:21:00,30171.348567,39989.599707,NaN,70160.948274
2022-10-31 09:22:00,29985.878497,NaN,30097.264775,60083.143272
2022-10-31 09:24:00,29995.763549,NaN,NaN,29995.763549
2022-10-31 09:25:00,30019.300345,NaN,NaN,30019.300345
...,...,...,...,...
2022-11-03 15:26:00,28872.588319,39361.510522,29923.033242,98157.132083
2022-11-03 15:27:00,28833.987628,NaN,NaN,28833.987628
2022-11-03 15:28:00,28831.633086,39362.667285,NaN,68194.300372


In [ ]:
def interactive_plot(df,title):
  fig = px.line(title = title)
  for i in df.columns:
    fig.add_scatter(x = df.index, y=df[i], name = i)
  fig.show()

In [ ]:
interactive_plot(portfolio_df,"Individual Asset Allocation Performance")

In [ ]:
  interactive_plot(returns_df,"Individual Asset Returns")

In [ ]:
cumulative_return = 100*(portfolio_df.Total[-1]/portfolio_df.Total[0]-1)
print("Portfolio's cumulative return was {} %".format(cumulative_return))

Portfolio's cumulative return was 230.238039333513 %


In [ ]:
portfolio_df['Daily_Return'] = portfolio_df['Total'].pct_change(1)
portfolio_df

,ASIANPAINT.NS,HDFCBANK.NS,REDINGTON.NS,HINDUNILVR.NS,PIDILITIND.NS,Total,Daily_Return
Date,,,,,,,
2015-11-02,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,100000.000000,NaN
2015-11-03,19820.601245,19978.880713,19823.275599,20093.008891,20152.773583,99868.540031,-0.001315
2015-11-04,19465.414917,19941.234310,20310.344696,20072.409336,19912.200901,99701.604161,-0.001672
2015-11-05,19419.059651,19818.193893,20379.310476,20020.599556,19433.689972,99070.853547,-0.006326
2015-11-06,19381.735280,19848.035731,20017.241116,20046.816479,19108.829340,98402.657947,-0.006745
...,...,...,...,...,...,...,...
2022-10-24,75512.010048,53538.095448,48318.965517,64709.116046,93217.441109,335295.628168,0.001761
2022-10-25,74876.888733,53476.573558,47922.413267,63563.044973,92195.447981,332034.368512,-0.009727
2022-10-27,74245.380071,53603.287041,47215.518294,62584.269663,91110.237015,328758.692084,-0.009865


In [ ]:
def annualized_sharpe_ratio(df):
  sharpe = df['Daily_Return'].mean()/df['Daily_Return'].std()
  annualized = sharpe*(252**0.5)
  return annualized

In [ ]:
portfolio_sharpe = annualized_sharpe_ratio(portfolio_df)

if cumulative_return > 10 and portfolio_sharpe>1.2:
  print("Great Job! your portfolio returns {}% and it's sharpe ratio is {}".format(cumulative_return,portfolio_sharpe))
else:
    print("You should reallocate,  your portfolio returns {}% and it's sharpe ratio is {}".format(cumulative_return,portfolio_sharpe))


Great Job! your portfolio returns 230.238039333513% and it's sharpe ratio is 1.2624550181843557
